# [문제]
- session id 부여

In [ ]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()


True

In [ ]:

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
    print(law[:50])

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]
prompt = ChatPromptTemplate.from_messages(template)

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    # model='gpt-4.1-nano',
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    # max_tokens=512,
)
## 6. trim 설정
trimmer = trim_messages(
    max_tokens=60,            ## 최대 토큰 수 제한
    strategy='last',          ## 최근 메세지부터 시작해서 자르기
    token_counter=llm,        ## 모델 기준으로 토큰 수 계산 
    include_system=True,      ## 시스템 메세지 포함
    allow_partial=False,      ## 메세지 자르기 제한
    start_on='human',         ## human 메세지 부터 자르기
)

## 7. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
) 

## 8. 대화 이력(히스토리) 관리 인스턴스 생성
history = InMemoryChatMessageHistory()
store = {}

## 9. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return history

## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행 (멀티턴)
while True:
    query = input('육아휴직과 관련된 질문하세요 >> ')
    if query.upper() == 'S':
        break
    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )
    print('\n' + '=' * 50 + '\n')


남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 20


In [ ]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='오늘 너 많이 바쁘냐? 느리네', additional_kwargs={}, response_metadata={}), AIMessage(content='죄송합니다. 현재 다소 많은 요청이 처리되고 있어 응답이 지연되고 있습니다. 제가 도와드릴 수 있는 육아휴직 관련한 질문이나 궁금한 사항이 있다면 말씀해 주시면 최선을 다해 빠르게 답변드리겠습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run-95449ee4-dce2-48b2-9780-c758b9c4fbeb-0'), HumanMessage(content='육아휴직 oecd 중 우리나라 몇 번째로 좋아?', additional_kwargs={}, response_metadata={}), AIMessage(content='제공된 법률 문서에는 대한민국의 육아휴직 제도에 대한 구체적인 국제 순위나 OECD 내에서의 위치에 대한 정보가 포함되어 있지 않습니다. 육아휴직에 대한 국제 비교나 순위를 알고 싶다면 OECD 보고서나 국제 노동 기구의 자료를 참조하시는 것이 좋습니다. 대한민국의 육아휴직 법률에 대한 내용을 알고 싶으시다면, "남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률"의 제19조를 참고하시면 됩니다. 이 조항에 따르면, 사업주는 근로자가 육아휴직을 신청할 경우 이를 허용해야 하며, 육아휴직의 기간은 1년 이내로, 특정 조건을 만족할 경우 추가 6개월까지 사용할 수 있습니다. (제19조)', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_f

In [ ]:
store


{'abc': InMemoryChatMessageHistory(messages=[])}

In [ ]:
store.keys()

dict_keys(['abc'])

In [ ]:
for message in get_history('abc').messages:
    print(f'[{message.type.upper()}]: {message.content}')

In [ ]:
## 질문
'''육아휴직 기간은?
육아휴직 조건은?
육아휴직 대상자는?
육아휴직 수당은? '''
# 첫 번째 질문이 뭐야?
# 질문 정리해 줘
# 네 역할은?
# 육아휴직 질문에 대한 답변 정리해 줘